In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [32]:
df = pd.read_csv('../../data/Customer Churn Data.csv')
df['international plan'] = (df['international plan'] == 'yes').astype(int)
df['voice mail plan'] = (df['voice mail plan'] == 'yes').astype(int)
ohe = OneHotEncoder(sparse = False)
ohe_states = pd.DataFrame(ohe.fit_transform(pd.DataFrame(df['state'])), columns = ohe.get_feature_names())
df = pd.concat([df, ohe_states], axis = 1)
df = df.drop(['state', 'area code'], axis = 1)
df

account length phone number  international plan  voice mail plan  \
0                128     382-4657                   0                1   
1                107     371-7191                   0                1   
2                137     358-1921                   0                0   
3                 84     375-9999                   1                0   
4                 75     330-6626                   1                0   
...              ...          ...                 ...              ...   
3328             192     414-4276                   0                1   
3329              68     370-3271                   0                0   
3330              28     328-8230                   0                0   
3331             184     364-6381                   1                0   
3332              74     400-4344                   0                1   

      number vmail messages  total day minutes  total day calls  \
0                        25              265.1              110   
1                        26              161.6              123   
2                         0              243.4              114   
3                         0              299.4               71   
4                         0              166.7              113   
...                     ...                ...              ...   
3328                     36              156.2               77   
3329                      0              231.1               57   
3330                      0              180.8              109   
3331                      0              213.8              105   
3332                     25              234.4              113   

      total day charge  total eve minutes  total eve calls  ...  x0_SD  x0_TN  \
0                45.07              197.4               99  ...    0.0    0.0   
1                27.47              195.5              103  ...    0.0    0.0   
2                41.38              121.2              110  ...    0.0    0.0   
3                50.90               61.9               88  ...    0.0    0.0   
4                28.34              148.3              122  ...    0.0    0.0   
...                ...                ...              ...  ...    ...    ...   
3328             26.55              215.5              126  ...    0.0    0.0   
3329             39.29              153.4               55  ...    0.0    0.0   
3330             30.74              288.8               58  ...    0.0    0.0   
3331             36.35              159.6               84  ...    0.0    0.0   
3332             39.85              265.9               82  ...    0.0    1.0   

      x0_TX  x0_UT  x0_VA  x0_VT  x0_WA  x0_WI  x0_WV  x0_WY  
0       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
1       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
2       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
4       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
...     ...    ...    ...    ...    ...    ...    ...    ...  
3328    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3329    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0  
3330    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3331    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3332    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[3333 rows x 70 columns]

In [33]:
y = df['churn']
X = df.copy()
X.drop(['churn', 'phone number'], axis = 1, inplace = True)

In [34]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train)

ss = StandardScaler()
X_train1 = ss.fit_transform(X_train1)
X_test1 = ss.transform(X_test1)

In [42]:
estimators = [('knn', KNeighborsClassifier(n_neighbors = 20)),   
              ('rf', RandomForestClassifier(n_estimators = 100)),
              ('log', LogisticRegression(solver = 'liblinear')),
              ('grad', GradientBoostingClassifier())]
stack = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression(), cv = 5)
stack.fit(X_train1, y_train1);
stack.score(X_train1, y_train1)

0.9871931696905016

In [43]:
metrics(y_test1, stack.predict(X_test1))

Accuracy: 0.944
Precision: 0.9036144578313253
Recall: 0.7352941176470589
F1: 0.8108108108108109


In [19]:
def metrics(y_true, y_pred):
    print('Accuracy: ' + str(accuracy_score(y_true, y_pred)))
    print('Precision: ' + str(precision_score(y_true, y_pred)))
    print('Recall: ' + str(recall_score(y_true, y_pred)))
    print('F1: ' + str(f1_score(y_true, y_pred)))

In [45]:
for i in stack.estimators_:
    metrics(y_test1, i.predict(X_test1))

Accuracy: 0.8368
Precision: 0.0
Recall: 0.0
F1: 0.0
Accuracy: 0.9296
Precision: 0.9833333333333333
Recall: 0.5784313725490197
F1: 0.7283950617283951
Accuracy: 0.824
Precision: 0.43333333333333335
Recall: 0.2549019607843137
F1: 0.32098765432098764
Accuracy: 0.9408
Precision: 0.9113924050632911
Recall: 0.7058823529411765
F1: 0.7955801104972375


/Users/jarodc33/opt/anaconda3/envs/customerchurn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
